## SILVER TO GOLD LAYER

### Gold Layer - Trade History


In [54]:
# Import necessary libraries and utility functions
import pandas as pd
from common_utilities import Portfolio, global_path, logger

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [55]:
# Read the CSV file
try:
    df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

    # Convert 'datetime' to datetime type
    df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

    # Sort the DataFrame by 'datetime'
    df_trade_history = df_trade_history.sort_values(by="datetime")

    logger.info(
        f"Read SILVER Layer trade history data from: {global_path.tradehistory_silver_file_path}"
    )
except Exception as e:
    logger.error(
        f"Failed to read SILVER Layer trade history data due to error: {e}"
    )

2024-08-02T16:41:52Z - INFO - Read SILVER Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks and round necessary columns.


In [56]:


trades = [
    Trade(**trade_data).model_dump()
    for trade_data in df_trade_history.iloc[90:100].to_dict(orient="records")
]

df = pd.DataFrame(trades)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     10 non-null     datetime64[ns]
 1   exchange     10 non-null     object        
 2   segment      10 non-null     object        
 3   stock_name   10 non-null     object        
 4   scrip_code   10 non-null     object        
 5   side         10 non-null     object        
 6   amount       10 non-null     float64       
 7   quantity     10 non-null     float64       
 8   price        10 non-null     float64       
 9   expiry_date  8 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 932.0+ bytes


,datetime,exchange,segment,stock_name,scrip_code,side,amount,quantity,price,expiry_date
0,2024-07-01 13:45:26,FON,FO,NIFTY-CE-24050-04JUL2024,NIFTY,SELL,14373.75,75.00,191.65,2024-07-04
1,2024-07-01 13:45:26,FON,FO,NIFTY-CE-24050-04JUL2024,NIFTY,SELL,4792.50,25.00,191.70,2024-07-04
2,2024-07-02 00:00:00,BMF,MF,MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH,F769K01DM9,BUY,999.95,19.29,51.84,NaT
3,2024-07-02 00:00:00,BMF,MF,SBI-MAGNUM-TAXGAIN-SCHEME-DIR-GR,F200K01UM9,BUY,999.89,2.20,455.11,NaT
4,2024-07-02 09:45:41,FON,FO,NIFTY-PE-24150-04JUL2024,NIFTY,BUY,3125.00,25.00,125.00,2024-07-04
5,2024-07-02 09:53:05,FON,FO,NIFTY-PE-24150-04JUL2024,NIFTY,SELL,3508.75,25.00,140.35,2024-07-04
6,2024-07-02 12:44:33,FON,FO,NIFTY-CE-24000-04JUL2024,NIFTY,BUY,18155.00,100.00,181.55,2024-07-04
7,2024-07-02 14:54:13,FON,FO,NIFTY-CE-24000-04JUL2024,NIFTY,SELL,5027.50,25.00,201.10,2024-07-04
8,2024-07-02 14:54:13,FON,FO,NIFTY-CE-24000-04JUL2024,NIFTY,SELL,10052.50,50.00,201.05,2024-07-04
9,2024-07-02 14:54:13,FON,FO,NIFTY-CE-24000-04JUL2024,NIFTY,SELL,5027.50,25.00,201.10,2024-07-04


In [57]:
# Apply the trade logic to each row of the DataFrame
try:
    # Instantiate the Portfolio object
    portfolio = Portfolio()

    data = [
        portfolio.trade(row.to_dict()) for _, row in df_trade_history.iterrows()
    ]
    data += portfolio.check_expired_stocks()

    # Create a DataFrame from the processed data
    df_trade_history = pd.DataFrame(data)

    # Round the values in the columns to two decimal places
    df_trade_history = df_trade_history.round(2)

    # Data cleanup
    df_trade_history = df_trade_history.replace("nan", None)
    df_trade_history = df_trade_history.fillna(None)

except Exception as e:
    logger.error(f"Failed to process trade history data due to error: {e}")

2024-08-02T16:41:52Z - ERROR - Failed to process trade history data due to error: Must specify a fill 'value' or 'method'.


### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [58]:
# Save the result as a CSV file
try:
    # Sort the DataFrame by 'segment', 'stock_name', and 'datetime'
    df_trade_history = df_trade_history.sort_values(
        by=["segment", "stock_name", "datetime"]
    )
    # Select relevant columns
    df_trade_history = df_trade_history[
        [
            "datetime",
            "exchange",
            "segment",
            "stock_name",
            "scrip_code",
            "expiry_date",
            "side",
            "quantity",
            "price",
            "amount",
            "holding_quantity",
            "avg_price",
            "holding_amount",
            "pnl_amount",
            "pnl_percentage",
        ]
    ]

    df_trade_history.to_csv(global_path.tradehistory_gold_file_path, index=None)
    logger.info(
        "GOLD Layer CSV file for trade history successfully created at:"
    )
    logger.info(global_path.tradehistory_gold_file_path.resolve())
    # Log the DataFrame info
    df_trade_history.info()
except Exception as e:
    logger.error(f"Failed to save GOLD Layer CSV file due to error: {e}")

2024-08-02T16:41:52Z - INFO - GOLD Layer CSV file for trade history successfully created at:
2024-08-02T16:41:52Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\TradeHistory\TradeHistory_data.csv


<class 'pandas.core.frame.DataFrame'>
Index: 176 entries, 2 to 93
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          176 non-null    datetime64[ns]
 1   exchange          176 non-null    object        
 2   segment           176 non-null    object        
 3   stock_name        176 non-null    object        
 4   scrip_code        176 non-null    object        
 5   expiry_date       130 non-null    object        
 6   side              176 non-null    object        
 7   quantity          176 non-null    float64       
 8   price             176 non-null    float64       
 9   amount            176 non-null    float64       
 10  holding_quantity  176 non-null    float64       
 11  avg_price         176 non-null    float64       
 12  holding_amount    176 non-null    float64       
 13  pnl_amount        176 non-null    float64       
 14  pnl_percentage    176 non-null  